<a href="https://colab.research.google.com/github/GathN/Information-Retrieval-Project/blob/main/Semantic_Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import spacy
import string
import gensim
import operator
import re

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 62 bytes


In [ ]:
!kaggle datasets download -d canggih/indonesian-food-recipes
!unzip indonesian-food-recipes.zip

  0% 0.00/3.63M [00:00<?, ?B/s]
100% 3.63M/3.63M [00:00<00:00, 48.5MB/s]
Archive:  indonesian-food-recipes.zip
  inflating: dataset-ayam.csv        
  inflating: dataset-ikan.csv        
  inflating: dataset-kambing.csv     
  inflating: dataset-sapi.csv        
  inflating: dataset-tahu.csv        
  inflating: dataset-telur.csv       
  inflating: dataset-tempe.csv       
  inflating: dataset-udang.csv       


In [ ]:
ayam = pd.read_csv('dataset-ayam.csv')
ikan = pd.read_csv('dataset-ikan.csv')
kambing = pd.read_csv('dataset-kambing.csv')
sapi = pd.read_csv('dataset-sapi.csv')
tahu = pd.read_csv('dataset-tahu.csv')
telur = pd.read_csv('dataset-telur.csv')
tempe = pd.read_csv('dataset-tempe.csv')
udang = pd.read_csv('dataset-udang.csv')

ayam

,Title,Ingredients,Steps,Loves,URL
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,/id/resep/4473027-ayam-woku-manado
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,/id/resep/4471956-ayam-goreng-tulang-lunak
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,/id/resep/4473057-ayam-cabai-kawin
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,/id/resep/4473023-ayam-geprek
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,/id/resep/4427438-minyak-ayam
...,...,...,...,...,...
1911,Nugget ayam simpel,400 gram ayam giling--1 lembar roti tawar kupa...,"Haluskan bawang putih, merica, dan garam--Camp...",0,/id/resep/4308989-nugget-ayam-simpel
1912,Ukepan Ayam,1/2 kg ayam--6 siung bawang merah--6 siung baw...,"Haluskan semua bumbu kecuali lengkuas, serai, ...",7,/id/resep/4289253-ukepan-ayam
1913,Chicken Stroganoff with Mushroom,"Fillet ayam, cuci bersih & lumuri sedikit tepu...","Panaskan butter dan minyak goreng, masak ayam ...",3,/id/resep/4308032-chicken-stroganoff-with-mush...
1914,Sop Pangsit Ayam,1/4 kg dada ayam--1 buah jeruk nipis--2 cm Jah...,Cuci ayam dengan menggunakan jeruk nipis dan g...,3,/id/resep/4308088-sop-pangsit-ayam


In [ ]:
df = pd.concat([ayam, ikan, kambing, sapi, tahu, telur, tempe, udang], axis=0, ignore_index=True)

df.tail()

,Title,Ingredients,Steps,Loves,URL
15636,Udang saus mentega,1 kg udang segar--1 buah bawang bombay kecil--...,"Bersihkan udang, siapkan bahan bahan--Tumis ba...",46,/id/resep/3794442-udang-saus-mentega
15637,Jipang tumis udang,500 gr jipang(saya pk 6bh jipang muda yg kecil...,Tumis bawang merah bawang putih hingga harum.-...,6,/id/resep/3795267-jipang-tumis-udang
15638,Tumis Udang Tempe,3 siung bawang merah--3 siung bawang putih--2 ...,"Tiriskan bawang putih, bawang merah dan cabe l...",3,/id/resep/3794945-tumis-udang-tempe
15639,Tumis kacang panjang terong udang,"Stgah ikat kacang panjang,ku bli 3ribu d psr--...","Potong2 kacang panjang,terong cuci bersih sisi...",12,/id/resep/3794623-tumis-kacang-panjang-terong-...
15640,Siomay Udang Ayam,3 bh dada ayam--1/2 kg udang bersih--6 bh bawa...,Giling ayam dan udang serta haluskan bawang pu...,28,/id/resep/3794009-siomay-udang-ayam


In [ ]:
index = [i for i in range(0,15641)]
index = pd.DataFrame(data=index)
df["id"]= index

In [ ]:
df

,Title,Ingredients,Steps,Loves,URL,id
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,/id/resep/4473027-ayam-woku-manado,0
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,/id/resep/4471956-ayam-goreng-tulang-lunak,1
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,/id/resep/4473057-ayam-cabai-kawin,2
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,/id/resep/4473023-ayam-geprek,3
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,/id/resep/4427438-minyak-ayam,4
...,...,...,...,...,...,...
15636,Udang saus mentega,1 kg udang segar--1 buah bawang bombay kecil--...,"Bersihkan udang, siapkan bahan bahan--Tumis ba...",46,/id/resep/3794442-udang-saus-mentega,15636
15637,Jipang tumis udang,500 gr jipang(saya pk 6bh jipang muda yg kecil...,Tumis bawang merah bawang putih hingga harum.-...,6,/id/resep/3795267-jipang-tumis-udang,15637
15638,Tumis Udang Tempe,3 siung bawang merah--3 siung bawang putih--2 ...,"Tiriskan bawang putih, bawang merah dan cabe l...",3,/id/resep/3794945-tumis-udang-tempe,15638
15639,Tumis kacang panjang terong udang,"Stgah ikat kacang panjang,ku bli 3ribu d psr--...","Potong2 kacang panjang,terong cuci bersih sisi...",12,/id/resep/3794623-tumis-kacang-panjang-terong-...,15639


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 2.9 MB/s 
     |████████████████████████████████| 3.4 MB 11.1 MB/s 
     |████████████████████████████████| 3.3 MB 47.8 MB/s 
     |████████████████████████████████| 1.2 MB 52.0 MB/s 
     |████████████████████████████████| 61 kB 532 kB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=f0b51fcb0d417f782cb11faf472e45df4bfc76bcb7c3a664f6e2ac60e556d501
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!apt install libomp-dev
# !pip install faiss-gpu
!python -m pip install --upgrade faiss faiss-cpu
!python -m pip install --upgrade faiss faiss-gpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (265 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [ ]:
import faiss
import numpy as np
import pickle
from pathlib import Path
import torch
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

cpu


In [ ]:
embeddings = model.encode(df.Title.to_list(), show_progress_bar=True)

Batches:   0%|          | 0/489 [00:00<?, ?it/s]

In [ ]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

Shape of the vectorised abstract: (768,)


In [ ]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.index.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 15641


In [ ]:
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[15640]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 0.0, 0.0, 17.81776237487793, 21.454849243164062, 21.454849243164062, 21.454849243164062, 21.454849243164062, 21.454849243164062, 21.454849243164062]

MAG paper IDs: [14534, 15640, 15612, 15218, 173, 14463, 1509, 13702, 13676, 180]


In [ ]:
!git clone https://github.com/kstathou/vector_engine

Cloning into 'vector_engine'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 74 (delta 32), reused 59 (delta 18), pack-reused 0
Unpacking objects: 100% (74/74), done.


In [ ]:
cd "vector_engine"

/content/vector_engine


In [ ]:
# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
project_dir = Path('notebooks').resolve().parents[0]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

/content/vector_engine


In [ ]:
# Used to do vector searches and display the results.
from vector_engine.utils import vector_search, id2details

In [ ]:
D, I = vector_search(["teriyaki"], model, index, num_results=10)

In [ ]:
print( [list(df[df.id == idx]["URL"]) for idx in I[0]])

[['/id/resep/3273930-kambing-teriyaki'], ['/id/resep/4376167-tahu-teriyaki'], ['/id/resep/4302802-dori-fillet-teriyaki'], ['/id/resep/3455387-rolade-sapi-teriyaki-%F0%9F%98%8B'], ['/id/resep/3952550-sapi-teriyaki'], ['/id/resep/4283078-sapi-teriyaki'], ['/id/resep/3280283-iga-kambing-teriyaki'], ['/id/resep/2758386-teriyaki-kambing'], ['/id/resep/4372653-tahu-susu-teriyaki'], ['/id/resep/4352215-kakap-fillet-teriyaki']]


In [ ]:
# Fetch the paper titles based on their index
id2details(df, I, "Title")

[['Kambing Teriyaki'],
 ['Tahu teriyaki'],
 ['Dori Fillet Teriyaki'],
 ['Rolade sapi teriyaki 😋'],
 ['Sapi Teriyaki'],
 ['Sapi Teriyaki'],
 ['Iga Kambing teriyaki'],
 ['Teriyaki kambing'],
 ['Tahu susu teriyaki'],
 ['Kakap Fillet Teriyaki']]

In [ ]:
df.loc[df['Title']=='Kambing Teriyaki']

,Title,Ingredients,Steps,Loves,URL,id
4355,Kambing Teriyaki,250 gram daging kambing potong dadu--1 bh toma...,"Iris tipis bahan seperti, bawang merah putih b...",2,/id/resep/3273930-kambing-teriyaki,4355


In [ ]:
!pip install gradio

     |████████████████████████████████| 982 kB 5.0 MB/s 
     |████████████████████████████████| 206 kB 51.3 MB/s 
     |████████████████████████████████| 2.0 MB 40.1 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 961 kB 47.8 MB/s 
     |████████████████████████████████| 3.6 MB 17.8 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=d59fe308543ca0a55ab263105b8a389409614ad3c24112ad8c36e2a8a1810146
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=c94341f756d5aba15a51fa27c7ee6c581556fa4a3e539b4b0c65ef3a182f24ce
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


In [ ]:
from bs4 import BeautifulSoup
import requests
from PIL import Image
from io import BytesIO
def flatten(t) :
  return [item for sublist in t for item in sublist]

In [ ]:
agent = str(input())

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36


In [ ]:
#headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
headers = {'user-agent' : agent}
html_page = requests.get("https://cookpad.com/id/resep/4357849-chiken-katsu-kriuk", headers=headers)
soup = BeautifulSoup(html_page.content, 'html.parser')
warning = soup.find("meta", itemprop="image")
if (warning is None):
  img_link = "saved!"
#img_link = warning.attrs['content']
#response = requests.get(img_link)
#img = Image.open(BytesIO(response.content))
#print(img_link)

In [ ]:
def recipe_search(search):
  D, I = vector_search([search], model, index, num_results=10)
  z = [list(df[df.id == idx]["Title"]) for idx in I[0]]#id2details(df, I, "Title")
  z = flatten(z)
  loves = [list(df[df.id == idx]["Loves"]) for idx in I[0]]
  loves = flatten(loves)
  URL = [list(df[df.id == idx]["URL"]) for idx in I[0]]
  URL = flatten(URL)
  result = []
  img = []
  text = []
  headers = {'user-agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 [FBAN/FBIOS;FBDV/iPhone10,4;FBMD/iPhone;FBSN/iOS;FBSV/13.3.1;FBSS/2;FBID/phone;FBLC/en_US;FBOP/5;FBCR/]'}
  for i in range(len(z)):
    #temp = str(loves[i]) + " " + z[i] + " " + URL[i]
    temp = "\N{growing heart}" + str(loves[i]) + " " + z[i]
    text.append(temp)
    if i < 3 :
      html_page = requests.get("http://cookpad.com"+URL[i], headers=headers)
      soup = BeautifulSoup(html_page.content, 'html.parser')
      warning = soup.find("meta", itemprop="image")
      img_link = warning.attrs['content']
      response = requests.get(img_link)
      img.append(Image.open(BytesIO(response.content)))
      result.append([temp, img[i]])
  return "\n".join(text), result

In [ ]:
import gradio as gr


iface = gr.Interface(
    recipe_search,
    inputs=["text"],
    outputs=["text", gr.outputs.Carousel(["text", "image"])]
    )
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://14841.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


KeyboardInterrupt: ignored

In [ ]:
import gradio as gr

def recipe_search(search):
  D, I = vector_search([search], model, index, num_results=10)
  # Implement Q&A model here...
  # Return answer as string
  z = [list(df[df.id == idx]["Title"]) for idx in I[0]]#id2details(df, I, "Title")
  z = flatten(z)
  return("\n".join(z))

iface = gr.Interface(fn=recipe_search, inputs=["text"], outputs="text").launch()

In [ ]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
import anvil.server

anvil.server.connect("5SW5LX6ECOWNCG6HNJILYQO2-U5X4MPJCFPKXA6WK")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
@anvil.server.callable
def recipe(search):
  D, I = vector_search([search], model, index, num_results=10)
  z = [list(df[df.id == idx]["Title"]) for idx in I[0]]#id2details(df, I, "Title")
  z = flatten(z)
  loves = [list(df[df.id == idx]["Loves"]) for idx in I[0]]
  loves = flatten(loves)
  URL = [list(df[df.id == idx]["URL"]) for idx in I[0]]
  URL = flatten(URL)
  Ingredients = [list(df[df.id == idx]["Ingredients"]) for idx in I[0]]
  Ingredients = flatten(Ingredients)
  Steps = [list(df[df.id == idx]["Steps"]) for idx in I[0]]
  Steps = flatten(Steps)
  result = []
  img = []
  text = []
  link = []
  ingredients = []
  steps = []
  headers = {'user-agent':agent}
  for i in range(len(z)):
    #temp = str(loves[i]) + " " + z[i] + " " + URL[i]
    temp = "\N{growing heart}" + str(loves[i]) + " " + z[i]
    text.append(temp)
    link.append("http://cookpad.com"+URL[i])
    if i < 3 :
      ingredients.append(Ingredients[i])
      steps.append(Steps[i])
      html_page = requests.get("http://cookpad.com"+URL[i], headers=headers)
      soup = BeautifulSoup(html_page.content, 'html.parser')
      warning = soup.find("meta", itemprop="image")
      if (warning is None):
        img_link = " "
      else :
        img_link = warning.attrs['content']
      img.append(img_link)
  return link, text, img, ingredients, steps